# ARCH-type models (package [`sktime`](https://www.sktime.net/en/stable/index.html))  

In [ ]:
import numpy as np
import pandas as pd

from sktime.forecasting.arch import ARCH
from sktime.utils.plotting import plot_series
# временной горизонт для прогнозирования
from sktime.forecasting.base import ForecastingHorizon

import pandas_datareader.data as web

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)
# Не показывать ValueWarning, ConvergenceWarning из statsmodels
# from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning
# warnings.simplefilter('ignore', category=ValueWarning)
# warnings.simplefilter('ignore', category=ConvergenceWarning)

## Fitting of (*)ARCH of given order

Let's load from [`FRED`](https://fred.stlouisfed.org/) weekly data on Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity (Symbol [`WGS10YR`](https://fred.stlouisfed.org/series/WGS10YR)) from 2000-01-01 to 2023-12-31 and let `y` be its first difference

In [ ]:
rate = web.DataReader(name='WTB3MS', data_source='fred', start='2000-01-01', end='2023-12-31')
y = rate.diff().dropna()

y.plot()
plt.show()

Let's fit AR(1)-GARCH(1,1)

Model's specification

\begin{align*}
	y_t&=\alpha_0+\phi_1y_{t-1}+u_t \\ u_t&=\sigma_t \epsilon_t \\
	\sigma^\lambda_t &=\omega+\alpha|\epsilon_{t-1}|^\lambda+\beta\sigma^\lambda_{t-1}
\end{align*}

где $\lambda=2$

In [ ]:
forecaster = ARCH(mean='AR', lags=1, vol='GARCH', p=1, q=1, power=2)  
forecaster.fit(y)

forecaster.summary()

In [ ]:
forecaster.get_fitted_params()

## Forecasting

Let's evaluate 5-periods forecasts for AR(1)-GARCH(1,1)

Model's specification

\begin{align*}
	y_t&=\alpha_0+\phi y_{t-1}+u_t \\ u_t&=\sigma_t \epsilon_t \\
	\sigma^\lambda_t &=\omega+\alpha |\epsilon_{t-1}|^\lambda+\beta\sigma^\lambda_{t-1}
\end{align*}

where $\lambda=2$

In [ ]:
forecaster = ARCH(mean='AR', lags=1, vol='GARCH', p=1, q=1, power=2)

# we set forecasting horizon
fh = ForecastingHorizon(np.arange(1,6), freq ='W-Fri')

y_pred = forecaster.fit_predict(y=y, fh=fh)

y_pred

In [ ]:
# forecasts of volatility
forecaster.predict_var(fh=fh)